In [1]:
import os
import cv2
import pickle as pkl
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
from torchvision.models import resnet50, ResNet50_Weights

In [2]:
from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [12]:
class LazyLoadDataset(Dataset):
  def __init__(self, path, train=True, transform=None):
    self.transform = transform
    self.train = train
    path = path + ('train/' if train else 'test/')

    self.pathX = path + 'X/'
    self.pathY = path + 'Y/'
    self.data = os.listdir(self.pathX)

  def __getitem__(self, idx):
    f = self.data[idx]

    img0 = cv2.imread(self.pathX + f + '/rgb/0.png')
    img1 = cv2.imread(self.pathX + f + '/rgb/1.png')
    img2 = cv2.imread(self.pathX + f + '/rgb/2.png')

    if self.transform:
      img0 = self.transform(img0)
      img1 = self.transform(img1)
      img2 = self.transform(img2)

    img0 = img0.reshape(3,224,224)
    img1 = img1.reshape(3,224,224)
    img2 = img2.reshape(3,224,224)

    depth = np.load(self.pathX + f + '/depth.npy')
    field_id = pkl.load(open(self.pathX + f + '/field_id.pkl', 'rb'))
    if self.train:
      Y = np.load(self.pathY + f + '.npy')
      return (img0.astype(np.float32), img1.astype(np.float32), img2.astype(np.float32), depth.astype(np.float32), field_id), Y.astype(np.float32)
    return (img0.astype(np.float32), img1.astype(np.float32), img2.astype(np.float32), depth.astype(np.float32), field_id)


  def __len__(self):
    return len(self.data)

In [4]:
num_epochs = 10
learning_rate = 0.001
batch_size = 64
num_classes = 12

In [5]:
weights = ResNet50_Weights.DEFAULT
model = resnet50(weights=weights)
model.fc = nn.Linear(2048, num_classes) # change fc for our data

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [6]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [7]:
weights = ResNet50_Weights.DEFAULT
preprocess = weights.transforms()

In [13]:
dataset = LazyLoadDataset('gdrive/MyDrive/ml_capstone/lazydata/')
print(len(dataset))

3396


In [14]:
train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [21]:
def train():
  model.train()
  for batch, (X, y) in enumerate(train_dataloader):

    size = len(train_dataloader.dataset)

    X = torch.stack(X[:3])

    pred = model(X[0,:,:])

    loss = loss_fn(pred, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    current = batch*len(X)
    if current % 283 == 0: # 12, actually did the wrong number so i only saw 0/ lol
      loss = loss.item()
      print(f'loss: {loss:>7f} [{current:>5d}/{size:>5d}]')


In [22]:
num_epochs = 1
for i in range(num_epochs):
  print(f'Epoch {i+1}\n---------------------------')
  train()

Epoch 1
---------------------------
loss: 0.000446 [    0/ 3396]


In [23]:
test_dataset = LazyLoadDataset('gdrive/MyDrive/ml_capstone/lazydata/', train=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [25]:
# with torch.no_grad():
#     for X in test_dataloader:
#       print(X[-1])
#       print(len(X[-1]))
#       break

In [26]:
outfile = 'gdrive/MyDrive/ml_capstone/submission1.csv'

output_file = open(outfile, 'w')

titles = ['ID', 'FINGER_POS_1', 'FINGER_POS_2', 'FINGER_POS_3', 'FINGER_POS_4', 'FINGER_POS_5', 'FINGER_POS_6',
         'FINGER_POS_7', 'FINGER_POS_8', 'FINGER_POS_9', 'FINGER_POS_10', 'FINGER_POS_11', 'FINGER_POS_12']
preds = []
file_ids = []

model.eval()

with torch.no_grad():
    for X in test_dataloader:
      file_ids += X[-1]
      X = torch.stack(X[:3])
      try:
        pred = model(X[0,:,:].reshape((64,3,224,224)))
      except:
        pred = model(X[0,:,:].reshape((17,3,224,224)))
      # print(pred)
      for inp in pred:
        preds.append(inp.cpu().detach().numpy())
      
# print(preds)
df = pd.concat([pd.DataFrame(file_ids), pd.DataFrame.from_records(preds)], axis = 1, names = titles)
df.columns = titles
df.to_csv(outfile, index = False)
print("Written to csv file {}".format(outfile))


Written to csv file gdrive/MyDrive/ml_capstone/submission1.csv
